In [54]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [55]:
#@title Imports all libraries
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [56]:
#@title Config Authorize Google Drive API
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive"]
TOKEN_FILE = "token.json"
CREDENTIALS_FILE = "credentials.json"
FOLDER_ID = "1-3y0hToO4S8KhoWzlrqcGwjOsBWOKvoX"

In [57]:
#@title Authorize Google Drive API
creds = None
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
    with open(TOKEN_FILE, "w") as token:
        token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=75969084923-cg6dock3ajhumvhg1p7ekrerkjdcls46.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A63368%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=AYCOxrjSb99X7AsDBOYCzoraBoDe0z&access_type=offline


In [58]:
#@title List files
service = build("drive", "v3", credentials=creds)
try:
    results = (
        service.files()
        .list(q=f"'{FOLDER_ID}' in parents and trashed = false", pageSize=10, fields="nextPageToken, files(id, name)")
        .execute()
    )
    items = results.get("files", [])
    if not items:
        print("No files found.")
    else:
        print("Files:")
        for item in items:
            print(f"{item['name']} ({item['id']})")
except HttpError as error:
    print(f"An error occurred: {error}")

Files:
snis-228_Mosaic.mp4 (1-Gksc1nqmUdfR1ER7HBcu0pgEqNkiB9Z)
MUDR-244.mp4 (1-5rLWa7CBwU6aTvIOkiLKjMR1MEkSoOi)
Free_Test_Data_15MB_MP4.mp4 (1-3goBUpthgyqVtFVFs_xc5aFmgapkDqL)


In [59]:
#@title copy local video to google drive
from googleapiclient.http import MediaFileUpload
local_video_path = "raw-video/statue.mp4"
file_metadata = {"name": "statue.mp4", "parents": [FOLDER_ID]}
media = MediaFileUpload(local_video_path, mimetype="video/mp4")
try:
    file = (
        service.files()
        .create(body=file_metadata, media_body=media, fields="id")
        .execute()
    )
    print(f"File ID: {file.get('id')}")
except HttpError as error:
    print(f"An error occurred: {error}")
    
    



File ID: 19RByzkcubZK_yZNmMviJKdRA4UNe16Y6
